In [8]:
import os
import json

# Rough ground for testing things out

In [9]:
test_file = './mlm_analysis/adv_dec/region_results.json'

In [10]:
with open(test_file, 'r') as f:
    data = json.load(f)

In [11]:
results = {}
scores = {}

In [12]:
for identity_term in data:
    results[identity_term] = {}
    scores[identity_term] = {}
    for subcategory in data[identity_term]:
        results[identity_term][subcategory] = {}
        scores[identity_term][subcategory] = {}
        for datum in data[identity_term][subcategory]:
            for top_results in datum:
                token = top_results['token_str']
                score = top_results['score']
                if token not in results[identity_term][subcategory]:
                    results[identity_term][subcategory][token] = 0
                    scores[identity_term][subcategory][token] = 0

                results[identity_term][subcategory][token] += 1
                scores[identity_term][subcategory][token] += score

In [13]:
# sort results by highest occurence
for identity_term in results:
    for subcategory in results[identity_term]:
        results[identity_term][subcategory] = {k: v for k, v in sorted(results[identity_term][subcategory].items(), key=lambda item: item[1], reverse=True)}
        scores[identity_term][subcategory] = {k: v for k, v in sorted(scores[identity_term][subcategory].items(), key=lambda item: item[1], reverse=True)}

# Compiling the results for each of the experiments

In [18]:
subcategory_map = {
    'professions': 'professions',
    'subjects_of_study': 'subjects',
    'action_verbs': 'verbs',
    'behaviour_adjectives': 'adjectives',
    'socio_economic_status_adjectives': 'socio_economic_status',
    'food_habits_adjectives': 'food',
    'clothing_preferences_adjectives': 'clothes',
    'general': 'general'
}

In [20]:
for folder in os.listdir('./mlm_analysis'):
    print(folder)
    os.makedirs(f'./mlm_analysis/{folder}/results', exist_ok=True)

    if 'combined' not in folder:
        files_list = ['region_results.json', 'religion_results.json']
        for file in files_list:
            file_path = os.path.join('./mlm_analysis', folder, file)
            with open(file_path, 'r') as f:
                data = json.load(f)

            results = {}
            scores = {}

            for identity_term in data:
                results[identity_term] = {}
                scores[identity_term] = {}
                for subcategory in data[identity_term]:
                    subcategory_term = subcategory_map[subcategory]

                    results[identity_term][subcategory_term] = {}
                    scores[identity_term][subcategory_term] = {}
                    for datum in data[identity_term][subcategory]:
                        for top_results in datum:
                            try:
                                token = top_results['token_str'].lower()
                                score = top_results['score']
                            except:
                                print(datum)

                            if token not in results[identity_term][subcategory_term]:
                                results[identity_term][subcategory_term][token] = 0
                                scores[identity_term][subcategory_term][token] = 0

                            results[identity_term][subcategory_term][token] += 1
                            scores[identity_term][subcategory_term][token] += score

            # sort results by highest occurence
            for identity_term in results:
                for subcategory in results[identity_term]:
                    results[identity_term][subcategory] = {k: v for k, v in sorted(results[identity_term][subcategory].items(), key=lambda item: item[1], reverse=True)}
                    scores[identity_term][subcategory] = {k: v for k, v in sorted(scores[identity_term][subcategory].items(), key=lambda item: item[1], reverse=True)}

            with open(f'./mlm_analysis/{folder}/results/{file}', 'w') as f:
                json.dump(results, f, indent=4)
            with open(f'./mlm_analysis/{folder}/results/scores_{file}', 'w') as f:
                json.dump(scores, f, indent=4)
    else:
        files_list = 'combined_results.json'
        file_path = os.path.join('./mlm_analysis', folder, files_list)
        with open(file_path, 'r') as f:
            data = json.load(f)

        results = {}
        scores = {}

        for identity_term_1 in data:
            results[identity_term_1] = {}
            scores[identity_term_1] = {}
            for identity_term_2 in data[identity_term_1]:
                results[identity_term_1][identity_term_2] = {}
                scores[identity_term_1][identity_term_2] = {}
                for subcategory in data[identity_term_1][identity_term_2]:
                    subcategory_term = subcategory_map[subcategory]
                    results[identity_term_1][identity_term_2][subcategory_term] = {}
                    scores[identity_term_1][identity_term_2][subcategory_term] = {}
                    for datum in data[identity_term_1][identity_term_2][subcategory]:
                        for top_results in datum:
                            token = top_results['token_str']
                            score = top_results['score']
                            if token not in results[identity_term_1][identity_term_2][subcategory_term]:
                                results[identity_term_1][identity_term_2][subcategory_term][token] = 0
                                scores[identity_term_1][identity_term_2][subcategory_term][token] = 0

                            results[identity_term_1][identity_term_2][subcategory_term][token] += 1
                            scores[identity_term_1][identity_term_2][subcategory_term][token] += score

        # sort results by highest occurence
        for identity_term_1 in results:
            for identity_term_2 in results[identity_term_1]:
                for subcategory in results[identity_term_1][identity_term_2]:
                    results[identity_term_1][identity_term_2][subcategory] = {k: v for k, v in sorted(results[identity_term_1][identity_term_2][subcategory].items(), key=lambda item: item[1], reverse=True)}
                    scores[identity_term_1][identity_term_2][subcategory] = {k: v for k, v in sorted(scores[identity_term_1][identity_term_2][subcategory].items(), key=lambda item: item[1], reverse=True)}

        with open(f'./mlm_analysis/{folder}/results/{files_list}', 'w') as f:
            json.dump(results, f, indent=4)
        with open(f'./mlm_analysis/{folder}/results/scores_{files_list}', 'w') as f:
            json.dump(scores, f, indent=4)

negative_framing
positive_framing
adv_inc
adv_perspective_shift
adv_dec
adv_future
vanilla
combined
adv_present
neutral_framing
adv_past
